In [ ]:
import pandas as pd

def calculate_and_compare_pnl(csv_file1, csv_file2):
    # Read the CSV files
    df1 = pd.read_csv(csv_file1)
    df2 = pd.read_csv(csv_file2)

    # Aggregate data in the first CSV
    aggregation_functions = {'Size': 'sum', 'MWH': 'sum', 'PNL': 'sum'}
    df1_aggregated = df1.groupby(['Trades', 'Region', 'Location', 'Option Type', 'Trade Price', 'Mark Price'], as_index=False).aggregate(aggregation_functions)

    # Prepare the second CSV for PNL calculation
    df2 = df2.merge(df1_aggregated[['Region', 'Location', 'Trade Price', 'Mark Price', 'Option Type']], on=['Region', 'Location', 'Trade Price'], how='left')

    # Define a function for PNL calculation based on Option Type
    def calculate_pnl(row):
        if row['Option Type'] == 'power':
            return row['Size'] * (row['Mark Price'] - row['Trade Price'])
        elif row['Option Type'] == 'gas':
            return row['Size'] * (row['Mark Price'] - row['Trade Price']) * 100
        else:
            return None  # or some default calculation

    # Apply the PNL calculation
    df2['Calculated PNL'] = df2.apply(calculate_pnl, axis=1)

    # Merge with the first CSV to compare PNL
    comparison = df2.merge(df1_aggregated[['Trades', 'PNL']], left_on='Trade', right_on='Trades', how='left')

    # Highlight differences in PNL
    comparison['PNL Difference'] = comparison['Calculated PNL'] - comparison['PNL']
    comparison['Highlight'] = comparison['PNL Difference'].apply(lambda x: 'RED' if x != 0 else '')

    return comparison[['Trade', 'PNL', 'Calculated PNL', 'Highlight']]

# Usage Example
# Replace 'csv1.csv' and 'csv2.csv' with your actual file paths
result = calculate_and_compare_pnl('csv1.csv', 'csv2.csv')
print(result)


In [ ]:
import pandas as pd

def calculate_and_compare_pnl(csv_file1, csv_file2):
    # Read the CSV files
    df1 = pd.read_csv(csv_file1)
    df2 = pd.read_csv(csv_file2)

    # Aggregate data in the first CSV
    aggregation_functions = {'Size': 'sum', 'MWH': 'sum', 'PNL': 'sum'}
    df1_aggregated = df1.groupby(['Region', 'Location', 'Option Type', 'Trade Price', 'Mark Price'], as_index=False).aggregate(aggregation_functions)

    # Merge df2 with the aggregated data from df1
    df2_merged = df2.merge(df1_aggregated[['Region', 'Location', 'Option Type', 'Trade Price', 'Mark Price', 'MWH']], on=['Region', 'Location', 'Option Type', 'Trade Price'], how='left')

    # Calculate PNL for df2
    def calculate_pnl(row):
        if row['Option Type'] == 'power':
            return row['Size'] * (row['Mark Price'] - row['Trade Price'])
        elif row['Option Type'] == 'gas':
            return row['Size'] * (row['Mark Price'] - row['Trade Price']) * 100
        else:
            return None  # or some default calculation
    df2_merged['Calculated PNL'] = df2_merged.apply(calculate_pnl, axis=1)

    # Compare PNLs
    compare_df = df2_merged.merge(df1_aggregated[['Region', 'Location', 'Option Type', 'Trade Price', 'PNL']], on=['Region', 'Location', 'Option Type', 'Trade Price'], how='outer', indicator=True)
    compare_df['Tieout'] = compare_df.apply(lambda row: 'Green' if row['Calculated PNL'] == row['PNL'] else 'Red', axis=1)
    
    # Handle unmatched trades
    compare_df['Tieout'] = compare_df.apply(lambda row: 'Unmatched' if row['_merge'] != 'both' else row['Tieout'], axis=1)

    return compare_df[['Trade', 'Region', 'Location', 'Option Type', 'Trade Price', 'PNL', 'Calculated PNL', 'Tieout']]

# Usage Example
# Replace 'csv1.csv' and 'csv2.csv' with your actual file paths
#result = calculate_and_compare_pnl('csv1.csv', 'csv2.csv')
#print(result)


In [ ]:
import pandas as pd

def append_matching_values(df1, df2):
    # Aggregate 'pnl' and 'quantity' in df1
    aggregated_df1 = df1.groupby(['Trade', 'Date', 'Price']).agg({'pnl': 'sum', 'quantity': 'sum'}).reset_index()

    # Initialize new columns in df2
    df2['matched_pnl'] = pd.NA
    df2['matched_quantity'] = pd.NA

    # Iterate through df2 rows
    for index, row in df2.iterrows():
        # Check if the row exists in aggregated_df1
        match = aggregated_df1[
            (aggregated_df1['Trade'] == row['Trade']) & 
            (aggregated_df1['Date'] == row['Date']) & 
            (aggregated_df1['Price'] == row['Price'])
        ]

        # If match is found, update the new columns in df2
        if not match.empty:
            df2.at[index, 'matched_pnl'] = match['pnl'].values[0]
            df2.at[index, 'matched_quantity'] = match['quantity'].values[0]

    return df2

# Example usage:
# df1 = your first DataFrame
# df2 = your second DataFrame
# result_df = append_matching_values(df1, df2)
